In [1]:
!pip install pandas torch transformers datasets scikit-learn rouge-score bert-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.7 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=26d7ca1626e9df264c695490ef92d8e8304c5c6a5b132989787989d797f640ef
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [2]:
import os
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from rouge_score import rouge_scorer
from bert_score import score as bert_score

In [3]:
# Load the dataset
input_path = '/kaggle/input/eco-news-toplu/eco_news_cleaned3.csv'
if not os.path.exists(input_path):
    raise FileNotFoundError(f"File not found: {input_path}")

# Read CSV file
df = pd.read_csv(input_path)

# Drop rows with NaN values in 'icerik' or 'ozet' columns
df = df.dropna(subset=['icerik', 'ozet'])

# Separate articles and summaries
articles = df['icerik'].tolist()
summaries = df['ozet'].tolist()

# Veriyi ilk olarak train ve test olarak ayırıyoruz
train_articles, test_articles, train_summaries, test_summaries = train_test_split(
    articles, summaries, test_size=0.2, random_state=42
)

# Train setini bir daha ayırarak train ve validation setlerini oluşturuyoruz
train_articles, val_articles, train_summaries, val_summaries = train_test_split(
    train_articles, train_summaries, test_size=0.1, random_state=42
)

print(f"Train set size: {len(train_articles)}")
print(f"Validation set size: {len(val_articles)}")
print(f"Test set size: {len(test_articles)}")

Train set size: 5576
Validation set size: 620
Test set size: 1550


In [4]:
# Her bir seti Hugging Face Dataset formatına dönüştürme
train_data = {'article': train_articles, 'summary': train_summaries}
val_data = {'article': val_articles, 'summary': val_summaries}
test_data = {'article': test_articles, 'summary': test_summaries}

train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)
test_dataset = Dataset.from_dict(test_data)

# DatasetDict formatında birleştirme
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

In [5]:
# Load tokenizer and model
model_name = 'google/flan-t5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [6]:
def preprocess_data(examples):
    inputs = [f"Özetle: {article}" for article in examples["article"]]
    targets = examples['summary']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=150, truncation=True, padding="max_length")
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
    
tokenized_dataset = dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/5576 [00:00<?, ? examples/s]

Map:   0%|          | 0/620 [00:00<?, ? examples/s]

Map:   0%|          | 0/1550 [00:00<?, ? examples/s]

In [7]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    save_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=20,
    weight_decay=0.01,
    predict_with_generate=True,
    logging_dir='./logs',
    report_to="none",
)

In [8]:
# Initialize Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
)

# Start training
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.552100,0.432722
2,0.459400,0.389785
3,0.416000,0.377458
4,0.397500,0.364673
5,0.392500,0.353719
6,0.381000,0.345619
7,0.370500,0.342900
8,0.349000,0.336619
9,0.342800,0.336293
10,0.328700,0.333982


TrainOutput(global_step=27880, training_loss=0.392600905365034, metrics={'train_runtime': 12564.873, 'train_samples_per_second': 8.876, 'train_steps_per_second': 2.219, 'total_flos': 7.636413994500096e+16, 'train_loss': 0.392600905365034, 'epoch': 20.0})

In [9]:
# Save the fine-tuned model
model.save_pretrained("fine_tuned_flan-t5-yeni")
tokenizer.save_pretrained("fine_tuned_flan-t5-yeni")

print("Fine-tuning complete. Model and tokenizer saved.")

Fine-tuning complete. Model and tokenizer saved.


In [15]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 5.8 MB/s eta 0:00:00


In [10]:
def generate_summary(model, tokenizer, text, max_length=150, min_length=30):
    """Haber metni için özet oluşturma."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    input_ids = tokenizer.encode(text, return_tensors="pt", max_length=512, truncation=True).to(device)
    summary_ids = model.generate(input_ids, max_length=max_length, min_length=min_length, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [11]:
# ROUGE Hesaplama Fonksiyonu
def evaluate_rouge(model, tokenizer):
    print("Calculating ROUGE scores...")
    rouge_results = []

    for article, reference_summary in zip(test_articles, test_summaries):
        predicted_summary = generate_summary(model, tokenizer, article)
        rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        scores = rouge.score(predicted_summary, reference_summary)
        rouge_results.append({
            "rouge1_f1": scores['rouge1'].fmeasure,
            "rouge2_f1": scores['rouge2'].fmeasure,
            "rougeL_f1": scores['rougeL'].fmeasure
        })

    avg_rouge = {
        key: sum(d[key] for d in rouge_results) / len(rouge_results)
        for key in rouge_results[0]
    }
    print(f"Average ROUGE Scores: {avg_rouge}")

# BLEU Hesaplama Fonksiyonu
def evaluate_bleu(model, tokenizer):
    print("Calculating BLEU scores...")
    bleu_results = []

    for article, reference_summary in zip(test_articles, test_summaries):
        predicted_summary = generate_summary(model, tokenizer, article)
        bleu_score = sacrebleu.sentence_bleu(predicted_summary, [reference_summary]).score
        bleu_results.append(bleu_score)

    avg_bleu = sum(bleu_results) / len(bleu_results)
    print(f"Average BLEU Score: {avg_bleu:.4f}")
    # Corpus BLEU Hesaplama Fonksiyonu

def evaluate_corpus_bleu(model, tokenizer):
    print("Calculating Corpus BLEU score...")
    predictions = []
    references = []

    for article, reference_summary in zip(test_articles, test_summaries):
        predicted_summary = generate_summary(model, tokenizer, article)
        predictions.append(predicted_summary)
        references.append([reference_summary])  # SacreBLEU çoklu referansı destekler, bu yüzden liste içinde olmalı

    bleu_score = sacrebleu.corpus_bleu(predictions, references).score
    print(f"Corpus BLEU Score: {bleu_score:.4f}")

# BERTScore Hesaplama Fonksiyonu
def evaluate_bertscore(model, tokenizer):
    print("Calculating BERTScore...")
    bert_results = []

    for article, reference_summary in zip(test_articles, test_summaries):
        predicted_summary = generate_summary(model, tokenizer, article)
        P, R, F1 = bert_score([predicted_summary], [reference_summary], lang="tr")
        bert_results.append(F1.mean().item())

    avg_bert = sum(bert_results) / len(bert_results)
    print(f"Average BERTScore: {avg_bert:.4f}")

In [12]:
evaluate_rouge(model, tokenizer)

Calculating ROUGE scores...
Average ROUGE Scores: {'rouge1_f1': 0.5448951185094446, 'rouge2_f1': 0.41971176557795264, 'rougeL_f1': 0.5222822915783296}


In [16]:
import sacrebleu
evaluate_bleu(model, tokenizer)

Calculating BLEU scores...
Average BLEU Score: 24.7775


In [18]:
evaluate_corpus_bleu(model, tokenizer)

Calculating Corpus BLEU score...
Corpus BLEU Score: 50.3175


In [17]:
evaluate_bertscore(model, tokenizer)

Calculating BERTScore...


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Average BERTScore: 0.7255
